# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [35]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})
print(movie_item)
for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

[<option data-name="兔嘲男孩" value="10283">兔嘲男孩</option>, <option data-name="海上鋼琴師4K數位修復" value="10425">海上鋼琴師4K數位修復</option>, <option data-name="鹿皮奇談" value="10420">鹿皮奇談</option>, <option data-name="大約在冬季" value="10408">大約在冬季</option>, <option data-name="倒數計死" value="10390">倒數計死</option>, <option data-name="吹哨人" value="10381">吹哨人</option>, <option data-name="幸福試用期" value="10374">幸福試用期</option>, <option data-name="竊竊屍語" value="10365">竊竊屍語</option>, <option data-name="舞力決戰" value="10344">舞力決戰</option>, <option data-name="潛艦追緝" value="10333">潛艦追緝</option>, <option data-name="霹靂嬌娃" value="10124">霹靂嬌娃</option>, <option data-name="白頭山：半島浩劫" value="10354">白頭山：半島浩劫</option>, <option data-name="CATS貓" value="10166">CATS貓</option>, <option data-name="金魚" value="10424">金魚</option>, <option data-name="海上花" value="10376">海上花</option>, <option data-name="奪魂露營車" value="10366">奪魂露營車</option>, <option data-name="怨靈古堡" value="10356">怨靈古堡</option>, <option data-name="添好孕" value="10332">添好孕</option>, <option

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [12]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10365

In [15]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
header = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=header)
print(resp.json())  # 若有需要，列印出json原始碼
print("=====================")
# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

[{'area_id': 28, 'title': '台北市', 'priority': 150, 'ts_update': 1547519309}, {'area_id': 10, 'title': '台南', 'priority': 100, 'ts_update': 1547507403}, {'area_id': 17, 'title': '高雄', 'priority': 95, 'ts_update': 1547507403}]
放映地區: 台北市, 代號(area_id): 28
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [16]:
# 指定放映地區
area_id = 28

In [22]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})
print(movie_date)
print("================")
# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

[<label class="picker_label" for="date_0">
<p class="month">一月</p>
<h3 class="day">1</h3>
<p class="week">今天</p>
</label>, <label class="picker_label" for="date_1">
<p class="month">一月</p>
<h3 class="day">2</h3>
<p class="week">明天</p>
</label>, <label class="picker_label" for="date_2">
<p class="month">一月</p>
<h3 class="day">3</h3>
<p class="week">禮拜五</p>
</label>, <label class="picker_label" for="date_3">
<p class="month">一月</p>
<h3 class="day">4</h3>
<p class="week">禮拜六</p>
</label>, <label class="picker_label" for="date_4">
<p class="month">一月</p>
<h3 class="day">5</h3>
<p class="week">禮拜天</p>
</label>]
一月 1
一月 2
一月 3
一月 4
一月 5


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [23]:
# 選定要觀看的日期
date = "2020-01-03"

In [44]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})
print(html)

[<ul class="area_time _c jq_area_time" data-theater_name="台北樂聲影城" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=38" data-theater_url="http://www.luxcinema.com.tw/" id="theater_id_38">
<li class="adds">
<a href="https://movies.yahoo.com.tw/theater_result.html/id=38">台北樂聲影城</a>
<span>02-23118628</span>
</li>
<li class="taps">
<span class="tapR">數位</span> </li>
<li class="time _c">
<div class="input_picker jq_input_picker">
<input class="gabtn" data-cinema="46efa950eafa11e38b12987bdb90dba4" data-date="1577980800" data-ezding="ezding" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '竊竊屍語']" data-movie="6da49f45cb6a409f93e1610df10b9315" data-movie_date="01.04" data-movie_time="02:10" data-movie_title="竊竊屍語" data-movie_type="數位" data-session="fbfa9180d7054c1b9c23043429695f0d" data-time="1578075000" id="72684796" name="schedule_list" type="radio" value="2020-01-04 02:10:00"/>
<label class="select" for="72684796">02:10</label>
</div>
</li>
</ul>]


In [52]:
for h in html:
    print('電影院：{}'.format(h['data-theater_name']))
    print('影片放映類型：{}'.format(h.find('span',class_='tapR').text))
    print('時間表：{}'.format([d['value'] for d in h.find_all('input', class_='gabtn')]))


電影院：台北樂聲影城
影片放映類型：數位
時間表：['2020-01-04 02:10:00']
